# Resolution Horizon Analysis for Visium HD

This notebook demonstrates how to perform **multi-resolution deconvolution** on Visium HD data using FlashDeconv.

**Key concept**: The "Resolution Horizon" is the critical bin size below which single-cell-level spatial information is preserved. Above this threshold, cell type signals rapidly collapse due to spatial averaging.

**What you'll learn**:
1. How to run FlashDeconv at multiple resolutions (8-128 μm)
2. How to identify the resolution horizon for your data
3. How to determine the optimal resolution for each cell type

## Part 1: Setup

### Install dependencies

In [ ]:
# Uncomment to install
# !pip install flashdeconv scanpy pyarrow

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.spatial import cKDTree
from scipy.stats import entropy
import time

from flashdeconv import FlashDeconv

# Publication-ready style
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.size': 9,
    'axes.linewidth': 0.8,
    'axes.labelsize': 10,
    'axes.titlesize': 11,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 8,
    'figure.dpi': 150,
})

### Download data

This example uses the **Visium HD Mouse Small Intestine** dataset from 10x Genomics.

**Spatial data**: https://www.10xgenomics.com/datasets/visium-hd-cytassist-gene-expression-libraries-of-mouse-intestine

**Reference data**: [Haber et al., Nature 2017](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92332) - Mouse intestinal epithelium scRNA-seq

```bash
# Download Visium HD data
curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.0.0/Visium_HD_Mouse_Small_Intestine/Visium_HD_Mouse_Small_Intestine_binned_outputs.tar.gz
tar -xzf Visium_HD_Mouse_Small_Intestine_binned_outputs.tar.gz

# Reference data: download from GEO GSE92332 or use pre-processed version
```

In [ ]:
# === CONFIGURE YOUR DATA PATHS HERE ===
VISIUM_HD_DIR = Path("./Visium_HD_Mouse_Small_Intestine_binned_outputs")
REFERENCE_PATH = Path("./haber_intestine_matched.h5ad")  # Your scRNA-seq reference
CELL_TYPE_KEY = "celltype"  # Column name in reference.obs containing cell type labels

## Part 2: Core Functions

These functions handle data loading, binning, and multi-resolution analysis.

In [ ]:
def load_visium_hd(data_dir: Path, bin_size: str = "016um"):
    """
    Load Visium HD data at specified resolution.
    
    Parameters
    ----------
    data_dir : Path
        Path to Visium HD binned outputs directory
    bin_size : str
        One of '002um', '008um', '016um'
    
    Returns
    -------
    adata : AnnData
        Spatial data with coordinates in .obsm['spatial']
    """
    import pyarrow.parquet as pq
    
    bin_path = data_dir / f"square_{bin_size}"
    adata = sc.read_10x_h5(bin_path / "filtered_feature_bc_matrix.h5")
    adata.var_names_make_unique()
    
    # Load spatial coordinates
    positions = pq.read_table(bin_path / "spatial" / "tissue_positions.parquet").to_pandas()
    positions = positions.set_index("barcode")
    common = adata.obs_names.intersection(positions.index)
    adata = adata[common].copy()
    adata.obsm["spatial"] = positions.loc[common, ["pxl_col_in_fullres", "pxl_row_in_fullres"]].values
    
    return adata


def aggregate_to_bin_size(adata, coords, target_um: int, original_um: int = 8):
    """
    Aggregate spots to coarser bin size.
    
    Parameters
    ----------
    adata : AnnData
        Fine-resolution spatial data
    coords : ndarray
        Spatial coordinates in pixels (n_spots, 2)
    target_um : int
        Target bin size in micrometers
    original_um : int
        Original bin size in micrometers
    
    Returns
    -------
    adata_agg : AnnData
        Aggregated data
    coords_agg : ndarray
        Aggregated coordinates
    """
    from scipy import sparse
    from scipy.spatial import cKDTree
    
    scale_factor = target_um // original_um
    
    if scale_factor == 1:
        return adata.copy(), coords.copy()
    
    # Estimate pixel spacing from nearest neighbor distances
    sample_size = min(10000, len(coords))
    tree = cKDTree(coords[:sample_size])
    distances, _ = tree.query(coords[:sample_size], k=2)
    pixel_spacing = np.median(distances[:, 1])
    
    # Grid size in pixels
    grid_size = pixel_spacing * scale_factor
    
    # Assign spots to grid cells
    min_coords = coords.min(axis=0)
    grid_x = ((coords[:, 0] - min_coords[0]) / grid_size).astype(int)
    grid_y = ((coords[:, 1] - min_coords[1]) / grid_size).astype(int)
    grid_ids = np.array([f"{x}_{y}" for x, y in zip(grid_x, grid_y)])
    
    # Aggregate
    unique_grids = np.unique(grid_ids)
    grid_to_idx = {g: i for i, g in enumerate(unique_grids)}
    
    # Build aggregation matrix
    row_indices = np.array([grid_to_idx[g] for g in grid_ids])
    col_indices = np.arange(len(grid_ids))
    
    agg_matrix = sparse.csr_matrix(
        (np.ones(len(grid_ids)), (row_indices, col_indices)),
        shape=(len(unique_grids), len(grid_ids))
    )
    
    # Sum counts
    if sparse.issparse(adata.X):
        new_X = agg_matrix @ adata.X
    else:
        new_X = agg_matrix @ adata.X
    
    # Compute centroid coordinates for each bin
    coords_agg = np.zeros((len(unique_grids), 2))
    for i, g in enumerate(unique_grids):
        mask = grid_ids == g
        coords_agg[i] = coords[mask].mean(axis=0)
    
    adata_agg = sc.AnnData(X=new_X, var=adata.var.copy())
    adata_agg.obsm["spatial"] = coords_agg
    
    return adata_agg, coords_agg

In [ ]:
def run_multiscale_analysis(adata_st, adata_ref, cell_type_key, 
                            bin_sizes=[8, 16, 32, 64, 128], base_um=8):
    """
    Run FlashDeconv at multiple resolutions.
    
    Parameters
    ----------
    adata_st : AnnData
        Spatial data at finest resolution
    adata_ref : AnnData
        Single-cell reference with cell type annotations
    cell_type_key : str
        Column in adata_ref.obs containing cell type labels
    bin_sizes : list
        List of bin sizes in micrometers
    base_um : int
        Base resolution of input data
    
    Returns
    -------
    results : dict
        {bin_size: {'proportions': ndarray, 'n_spots': int, 'runtime': float}}
    cell_types : list
        Cell type names
    """
    # Build signature matrix from reference
    cell_types = sorted(adata_ref.obs[cell_type_key].unique())
    
    # Align genes
    common_genes = adata_st.var_names.intersection(adata_ref.var_names)
    print(f"Common genes: {len(common_genes)}")
    
    adata_st = adata_st[:, common_genes].copy()
    adata_ref = adata_ref[:, common_genes].copy()
    
    # Build signature matrix (K x G)
    X_ref = np.zeros((len(cell_types), len(common_genes)))
    for i, ct in enumerate(cell_types):
        mask = adata_ref.obs[cell_type_key] == ct
        X_ref[i] = np.asarray(adata_ref[mask].X.mean(axis=0)).flatten()
    
    coords = adata_st.obsm["spatial"]
    results = {}
    
    for bin_size in bin_sizes:
        print(f"\n{'='*50}")
        print(f"Processing {bin_size} μm resolution...")
        
        # Aggregate if needed
        adata_bin, coords_bin = aggregate_to_bin_size(
            adata_st, coords, bin_size, base_um
        )
        print(f"  Spots: {adata_bin.n_obs:,}")
        
        # Get count matrix
        if hasattr(adata_bin.X, 'toarray'):
            Y = adata_bin.X.toarray()
        else:
            Y = np.asarray(adata_bin.X)
        
        # Run FlashDeconv
        model = FlashDeconv(
            sketch_dim=512,
            lambda_spatial=5000,
            n_hvg=2000,
            n_markers_per_type=50,
            verbose=False
        )
        
        t0 = time.time()
        props = model.fit_transform(Y, X_ref, coords_bin, cell_type_names=cell_types)
        runtime = time.time() - t0
        
        print(f"  Runtime: {runtime:.2f}s ({adata_bin.n_obs/runtime:,.0f} spots/sec)")
        
        results[bin_size] = {
            'proportions': props,
            'coords': coords_bin,
            'n_spots': adata_bin.n_obs,
            'runtime': runtime
        }
    
    return results, cell_types

## Part 3: Resolution Horizon Metrics

In [ ]:
def compute_resolution_metrics(results, cell_types):
    """
    Compute metrics that reveal the resolution horizon.
    
    Returns
    -------
    metrics : DataFrame
        Metrics for each bin size
    """
    records = []
    
    for bin_size, data in results.items():
        props = data['proportions']
        
        # 1. Signal purity: fraction of spots with >80% dominant cell type
        max_props = props.max(axis=1)
        purity = (max_props > 0.8).mean()
        
        # 2. Mixing entropy: average Shannon entropy across spots
        props_safe = np.clip(props, 1e-10, 1)
        spot_entropy = entropy(props_safe, axis=1, base=2)
        max_entropy = np.log2(props.shape[1])  # Normalize
        mean_entropy = spot_entropy.mean() / max_entropy
        
        # 3. Effective cell types per spot
        effective_k = np.exp(entropy(props_safe, axis=1)).mean()
        
        # 4. Per-cell-type max proportion (can we still detect each type?)
        ct_max = {ct: props[:, i].max() for i, ct in enumerate(cell_types)}
        
        records.append({
            'bin_size': bin_size,
            'n_spots': data['n_spots'],
            'runtime': data['runtime'],
            'purity': purity,
            'entropy': mean_entropy,
            'effective_k': effective_k,
            **{f'max_{ct}': v for ct, v in ct_max.items()}
        })
    
    return pd.DataFrame(records).sort_values('bin_size')

## Part 4: Visualization

In [ ]:
def plot_resolution_horizon(metrics, cell_types, highlight_types=None):
    """
    Create publication-ready resolution horizon figure.
    
    Parameters
    ----------
    metrics : DataFrame
        Output from compute_resolution_metrics
    cell_types : list
        Cell type names
    highlight_types : list, optional
        Cell types to highlight (e.g., rare types)
    """
    fig, axes = plt.subplots(1, 3, figsize=(10, 3.5))
    
    bin_sizes = metrics['bin_size'].values
    x = np.arange(len(bin_sizes))
    
    # Panel A: Signal purity collapse
    ax = axes[0]
    purity = metrics['purity'].values * 100
    bars = ax.bar(x, purity, color='#2ecc71', edgecolor='black', linewidth=0.5)
    
    # Highlight the "cliff"
    if len(purity) > 1:
        drop = purity[0] - purity[1]
        if drop > 20:
            ax.annotate(f'-{drop:.0f}%', xy=(0.5, purity[1] + drop/2),
                       fontsize=9, ha='center', color='red', weight='bold')
    
    ax.set_xticks(x)
    ax.set_xticklabels([f'{b}μm' for b in bin_sizes])
    ax.set_ylabel('Pure spots (>80% dominant type) [%]')
    ax.set_xlabel('Bin size')
    ax.set_title('A. Signal Purity', fontweight='bold', loc='left')
    ax.set_ylim(0, 100)
    ax.axhline(50, ls='--', color='gray', lw=0.8, alpha=0.5)
    
    # Panel B: Mixing entropy
    ax = axes[1]
    ent = metrics['entropy'].values
    ax.plot(x, ent, 'o-', color='#e74c3c', lw=2, markersize=8)
    ax.fill_between(x, 0, ent, alpha=0.2, color='#e74c3c')
    ax.set_xticks(x)
    ax.set_xticklabels([f'{b}μm' for b in bin_sizes])
    ax.set_ylabel('Normalized mixing entropy')
    ax.set_xlabel('Bin size')
    ax.set_title('B. Signal Mixing', fontweight='bold', loc='left')
    ax.set_ylim(0, 1)
    
    # Panel C: Cell type detectability
    ax = axes[2]
    cmap = plt.cm.tab10
    
    for i, ct in enumerate(cell_types):
        col = f'max_{ct}'
        if col in metrics.columns:
            vals = metrics[col].values * 100
            style = '-' if highlight_types and ct in highlight_types else '--'
            lw = 2 if highlight_types and ct in highlight_types else 1
            alpha = 1.0 if highlight_types and ct in highlight_types else 0.5
            ax.plot(x, vals, style, color=cmap(i % 10), lw=lw, alpha=alpha,
                   label=ct, marker='o', markersize=4)
    
    ax.set_xticks(x)
    ax.set_xticklabels([f'{b}μm' for b in bin_sizes])
    ax.set_ylabel('Max detectable proportion [%]')
    ax.set_xlabel('Bin size')
    ax.set_title('C. Cell Type Detectability', fontweight='bold', loc='left')
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=7)
    ax.set_ylim(0, 105)
    ax.axhline(10, ls=':', color='gray', lw=0.8, alpha=0.5)
    ax.text(len(x)-0.5, 12, 'detection limit', fontsize=7, color='gray')
    
    plt.tight_layout()
    return fig

## Part 5: Run Analysis

Execute the multi-resolution analysis on your data.

In [ ]:
# Load data
print("Loading Visium HD data...")
adata_st = load_visium_hd(VISIUM_HD_DIR, bin_size="008um")  # Start from 8μm
print(f"Spatial data: {adata_st.n_obs:,} spots, {adata_st.n_vars:,} genes")

print("\nLoading reference data...")
adata_ref = sc.read_h5ad(REFERENCE_PATH)
print(f"Reference: {adata_ref.n_obs:,} cells, {adata_ref.n_vars:,} genes")
print(f"Cell types: {adata_ref.obs[CELL_TYPE_KEY].nunique()}")

In [ ]:
# Run multi-resolution analysis
results, cell_types = run_multiscale_analysis(
    adata_st, 
    adata_ref, 
    cell_type_key=CELL_TYPE_KEY,
    bin_sizes=[8, 16, 32, 64, 128],  # Adjust based on your data
    base_um=8
)

print(f"\nCell types: {cell_types}")

In [ ]:
# Compute resolution metrics
metrics = compute_resolution_metrics(results, cell_types)
print("\n" + "="*60)
print("RESOLUTION HORIZON ANALYSIS")
print("="*60)
print(metrics[['bin_size', 'n_spots', 'runtime', 'purity', 'entropy', 'effective_k']].to_string(index=False))

In [ ]:
# Visualize
# Specify rare cell types to highlight (adjust for your data)
rare_types = [ct for ct in cell_types if 'stem' in ct.lower() or 'tuft' in ct.lower()]

fig = plot_resolution_horizon(metrics, cell_types, highlight_types=rare_types)
fig.savefig('resolution_horizon.pdf', bbox_inches='tight', dpi=300)
plt.show()

## Part 6: Key Findings

In [ ]:
def summarize_findings(metrics, cell_types):
    """Print key findings from the resolution analysis."""
    
    print("\n" + "="*60)
    print("KEY FINDINGS")
    print("="*60)
    
    # 1. Resolution horizon
    purity = metrics['purity'].values
    bin_sizes = metrics['bin_size'].values
    
    # Find where purity drops below 50%
    horizon_idx = np.where(purity < 0.5)[0]
    if len(horizon_idx) > 0:
        horizon = bin_sizes[horizon_idx[0]]
        print(f"\n1. RESOLUTION HORIZON: {bin_sizes[horizon_idx[0]-1]}μm → {horizon}μm")
        print(f"   Purity drops below 50% at {horizon}μm")
    
    # 2. Information loss
    if len(purity) > 1:
        loss = (purity[0] - purity[1]) / purity[0] * 100
        print(f"\n2. INFORMATION LOSS: {loss:.0f}% from {bin_sizes[0]}μm to {bin_sizes[1]}μm")
    
    # 3. Cell type resolution requirements
    print(f"\n3. CELL TYPE RESOLUTION SENSITIVITY:")
    for ct in cell_types:
        col = f'max_{ct}'
        if col in metrics.columns:
            vals = metrics[col].values
            # Find smallest bin size where max prop > 10%
            detectable = bin_sizes[vals > 0.1]
            if len(detectable) > 0:
                loss_pct = (vals[0] - vals[-1]) / vals[0] * 100 if vals[0] > 0 else 0
                status = "⚠️ SENSITIVE" if loss_pct > 50 else "✓ robust"
                print(f"   {ct:20s}: {loss_pct:5.0f}% signal loss  {status}")
    
    # 4. Recommendations
    print(f"\n4. RECOMMENDATIONS:")
    finest = bin_sizes[0]
    print(f"   • For rare cell types: use ≤{finest}μm resolution")
    print(f"   • For abundant cell types: {bin_sizes[min(2, len(bin_sizes)-1)]}μm may suffice")
    print(f"   • Always run multi-scale analysis to validate findings")

summarize_findings(metrics, cell_types)

---

## Adapting to Your Own Data

To use this analysis with your own Visium HD dataset:

1. **Prepare your spatial data**: Load as AnnData with coordinates in `.obsm['spatial']`

2. **Prepare your reference**: scRNA-seq AnnData with cell type labels in `.obs`

3. **Modify the config**:
```python
VISIUM_HD_DIR = Path("/path/to/your/visium_hd_outputs")
REFERENCE_PATH = Path("/path/to/your/reference.h5ad")
CELL_TYPE_KEY = "your_celltype_column"
```

4. **Adjust bin sizes** based on your platform:
   - Visium HD: `[8, 16, 32, 64, 128]`
   - Standard Visium: `[55, 110, 220]` (55μm native resolution)

---

## References

- FlashDeconv: https://github.com/cafferychen777/flashdeconv
- 10x Genomics Visium HD: https://www.10xgenomics.com/products/visium-hd-spatial-gene-expression
- Haber et al. (2017). A single-cell survey of the small intestinal epithelium. *Nature*, 551, 333-339.